# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

ModuleNotFoundError: No module named 'mplleaflet'

In [2]:
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106


In [3]:
df.Date = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [4]:
df['Year']=df.Date.dt.year
df['Month'] = df.Date.dt.month
df['Day']= df.Date.dt.day

In [5]:
df.head()

,ID,Date,Element,Data_Value,Year,Month,Day
0,USW00094889,2014-11-12,TMAX,22,2014,11,12
1,USC00208972,2009-04-29,TMIN,56,2009,4,29
2,USC00200032,2008-05-26,TMAX,278,2008,5,26
3,USC00205563,2005-11-11,TMAX,139,2005,11,11
4,USC00200230,2014-02-27,TMAX,-106,2014,2,27


In [6]:
#plt.plot(df['Data_value'].values,df['Date'].values, '--r')
#observation_dates = list(map(pd.to_datetime, df['Date'].values))

In [7]:
df1 = df[(df['Element'] == 'TMAX') & (df['Year'] != 2015)]
df2 = df[(df['Element'] == 'TMIN') & (df['Year'] != 2015)]

In [8]:
df1 = df1.groupby(['Month','Day'])
df2 = df2.groupby(['Month','Day'])

In [9]:
s1 = df1['Data_Value'].agg(['max'])
s2 = df2['Data_Value'].agg(['min'])
s1.drop((2,29),inplace=True)
s2.drop((2,29),inplace=True)

In [10]:
df3 = df[(df['Element'] == 'TMAX') & (df['Year'] == 2015)]
df4 = df[(df['Element'] == 'TMIN') & (df['Year'] == 2015)]

In [11]:
df3 = df3.groupby(['Month','Day'])
df4 = df4.groupby(['Month','Day'])
s3 = df3['Data_Value'].agg(['max'])
s4 = df4['Data_Value'].agg(['min'])

In [12]:
s5 = pd.merge(s1,s3,left_index=True,right_index=True)
s6 = pd.merge(s2,s4,left_index=True,right_index=True)
s5.head()
s5['flag']= s5['max_x'] < s5['max_y']
s6['flag']= s6['min_x'] > s6['min_y']

s5.reset_index(inplace=True)
s6.reset_index(inplace=True)
s1.reset_index(inplace=True)
s2.reset_index(inplace=True)

In [13]:
s5 = s5.loc[s5['flag'] == True]
s6 = s6.loc[s6['flag'] == True]

In [21]:
import matplotlib.patches as mpatches
#plt.style.use('seaborn')
plt.plot(s1['max'].values,label='MaxTemp',color='orange')
plt.plot(s2['min'].values,label='MinTemp',color='cyan')
plt.xlabel('Day of year')
plt.ylabel('Temp')
plt.gca().fill_between(range(len(s1['max'].values)), 
                       s1['max'].values, s2['min'].values, 
                       facecolor='yellow', 
                       alpha=0.25)
plt.scatter(s5.index,s5.max_y.values,c='red',label='Max 2015')
plt.scatter(s6.index,s6.min_y.values,c='blue',label='Min 2015')
plt.legend()

# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top='off', bottom='off', left='on', right='off', labelleft='on', labelbottom='off')

# remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.show()

In [22]:
plt.draw()
plt.savefig('ass2')